In [ ]:
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.llms import Ollama

# Configuración global del sistema
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")
llm = Ollama(model="llama3.2")

# Variables para almacenamiento persistente
vector_store = None
retriever = None

def process_pdf(pdf_file):
    """
    Procesa un archivo PDF y crea un vector store basado en los embeddings.
    """
    global vector_store, retriever

    # Cargar el PDF
    loader = PyPDFLoader(pdf_file.name)
    documents = loader.load()

    # Dividir texto en fragmentos
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_docs = text_splitter.split_documents(documents)

    # Crear embeddings para el vector store
    texts = [doc.page_content for doc in split_docs]
    vector_store = Chroma.from_texts(
        texts=texts,
        embedding=embeddings,
        persist_directory="./chroma_pdf_facts",
    )

    # Crear un retriever
    retriever = vector_store.as_retriever()

    return "PDF procesado exitosamente. ¡Ahora puedes hacer preguntas!"

def answer_question(question):
    """
    Responde una pregunta utilizando el contenido del vector store.
    """
    if not retriever:
        return "Por favor, carga primero un archivo PDF."

    # Recuperar contexto relevante
    results = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in results])

    # Crear el prompt
    template = """Responde en base al siguiente contexto:
    {context}

    Pregunta: {question}
    Respuesta: en español.
    """
    prompt = ChatPromptTemplate.from_template(template)
    final_prompt = prompt.format(context=context, question=question)

    # Obtener la respuesta del modelo
    response = llm(final_prompt)
    return response

# Crear la interfaz de Gradio
with gr.Blocks() as gui:
    gr.Markdown("# 📄 PDF Question Answering System")
    gr.Markdown("Carga un archivo PDF, realiza preguntas y obtén respuestas en español basadas en el contenido del documento.")

    with gr.Row():
        pdf_input = gr.File(label="Sube tu archivo PDF", type="filepath")
        pdf_status = gr.Label(label="Estado del PDF")

    process_button = gr.Button("Procesar PDF")
    process_button.click(process_pdf, inputs=pdf_input, outputs=pdf_status)

    with gr.Row():
        question_input = gr.Textbox(label="Pregunta", placeholder="Escribe tu pregunta aquí...")
        answer_output = gr.Textbox(label="Respuesta")

    ask_button = gr.Button("Responder")
    ask_button.click(answer_question, inputs=question_input, outputs=answer_output)

# Ejecutar la aplicación
if __name__ == "__main__":
    gui.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_39599/1308332642.py:52: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(question)
/tmp/ipykernel_39599/1308332642.py:66: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(final_prompt)
